In [1]:
from __future__ import print_function

from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

import numpy as np

Using Theano backend.


In [2]:
model = Sequential()
model.add(Dense(128, input_dim=20, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))
# model.add(Dense(1, activation='linear'))


In [3]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 128)           2688        dense_input_1[0][0]              
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 128)           256         dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 128)           0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 256)           33024       dropout_1[0][0]                  
___________________________________________________________________________________________

this if for training only. It's the name of the text file that contains both training and test set

In [5]:
dataNameTrain= "baptiste-reba3-train.csv"
dataNameTest = "baptiste-reba3-test.csv"


In [6]:
dataTrain = np.loadtxt(dataNameTrain, delimiter=",", skiprows=1)
dataTest = np.loadtxt(dataNameTest, delimiter=",", skiprows=1)

In [7]:
dataTrain[:3]

array([[ 4.        , -0.16282838,  0.76110785, -0.37865804,  0.59945448,
        -0.14792126,  0.30589173, -0.83224183, -0.92893148,  0.7219417 ,
         0.11748402,  0.40186744,  0.21904875,  2.32952007, -0.33009103,
        -0.55839365, -0.27316892,  0.49377564, -0.3668738 , -0.47996572,
         0.16550142],
       [ 8.        ,  0.03256526, -0.07450412,  0.47708814, -0.78527557,
        -0.25193403,  0.05555838, -0.46927532, -0.44193896,  1.37965883,
         0.53487032,  0.28550838, -1.90195344,  1.41046677, -0.18468289,
        -0.38005845, -0.44065509,  0.2681269 , -0.28123809, -0.35075261,
        -0.15652698],
       [ 6.        ,  0.42963129,  0.33180792,  0.75388141,  0.76094014,
         0.22545793,  0.75969206, -1.39389067, -0.07975607,  1.33865431,
         0.65349093,  0.43027813,  0.67353188,  0.11422972, -0.98957083,
        -0.70577845, -0.43705815,  0.09005189,  0.25503179, -0.35117124,
        -0.06272062]])

show the amount of data... should be N x 19, where N >= 1000

In [8]:
print (dataTrain.shape)
print (dataTest.shape)

(100000, 21)
(30000, 21)


In [7]:
rawDataTrain = dataTrain[:,1:]
rawLabelsTrain = dataTrain[:,0]

rawDataTest = dataTest[:,1:]
rawLabelsTest = dataTest[:,0]

In [8]:
nb_classes = 10 # 10 output reba values

In [9]:
indices = np.arange(len(rawLabelsTrain))
np.random.shuffle(indices)

split = 0.8 # how much of the data is training. split = 0.8 means 80% train, 20% test
splitIdx = int(round(len(rawLabelsTrain)*0.8))

X_train = rawDataTrain[indices[:splitIdx]]
y_train = rawLabelsTrain[indices[:splitIdx]]
y_train -= 1
Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_train = np.expand_dims(y_train, axis=1)

X_val = rawDataTrain[indices[splitIdx:]]
y_val = rawLabelsTrain[indices[splitIdx:]]
y_val -= 1
Y_val = np_utils.to_categorical(y_val, nb_classes)
# Y_val = np.expand_dims(y_val, axis=1)

X_test = rawDataTest
y_test = rawLabelsTest
y_test -= 1
Y_test = np_utils.to_categorical(y_test, nb_classes)
# Y_test = np.expand_dims(y_test, axis=1)

show the amount of test/training data

In [12]:
print (X_train.shape)
print (Y_train.shape)

print (X_val.shape)
print (Y_val.shape)

print (X_test.shape)
print (Y_test.shape)

(80000, 20)
(80000, 10)
(20000, 20)
(20000, 10)
(30000, 20)
(30000, 10)


some hyperparameters necessary for training

In [13]:
epochs = 50 # how many training iterations
batch_size = 64 # in each iteration, batch learning is used. This specifies how big the batch has to be

# val_split = 0.2 # validation split, how much of the training data is used for validation during training - 
# this is important, because the training data might overfit (the performance getting better), 
# but the validation split would then show the overall performance going down


This trains the network and shows a little progress bar for each epoch, 
takes about 5min on my Nvidia Quadro K5200

In [14]:
hist = model.fit(
    X_train, 
    Y_train, 
    nb_epoch=epochs, 
    batch_size=batch_size, 
    validation_data=(X_val, Y_val)
)


Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 17s - loss: 1.6401 - acc: 0.3500 - val_loss: 1.3713 - val_acc: 0.4336
Epoch 2/50
80000/80000 [==============================] - 16s - loss: 1.4106 - acc: 0.4211 - val_loss: 1.2183 - val_acc: 0.4920
Epoch 3/50
80000/80000 [==============================] - 16s - loss: 1.3171 - acc: 0.4561 - val_loss: 1.1268 - val_acc: 0.5346
Epoch 4/50
80000/80000 [==============================] - 16s - loss: 1.2389 - acc: 0.4875 - val_loss: 1.0750 - val_acc: 0.5638
Epoch 5/50
80000/80000 [==============================] - 16s - loss: 1.1789 - acc: 0.5134 - val_loss: 0.9992 - val_acc: 0.5883
Epoch 6/50
80000/80000 [==============================] - 16s - loss: 1.1297 - acc: 0.5360 - val_loss: 0.9584 - val_acc: 0.6140
Epoch 7/50
80000/80000 [==============================] - 16s - loss: 1.0886 - acc: 0.5541 - val_loss: 0.9166 - val_acc: 0.6293
Epoch 8/50
80000/80000 [==============================

#### run the model on the test data for evaluation

In [15]:
score = model.evaluate(X_test, Y_test, batch_size=batch_size)

29952/30000 [============================>.] - ETA: 0s

In [16]:
score # this gives the mean squared error over the test set

[0.58908208888371782, 0.76503333333333334]

In [17]:
assert False

AssertionError: 

In [ ]:
modelName = "baptiste-reba3"
modelVersion = 2

In [ ]:
modelName = "model-"+modelName+'-'+str(modelVersion)
model.save(modelName+'-full.h5')
model.save_weights(modelName+'-weights.h5')

# IMPORTANT: STARTING FROM THE NEXT CELL YOU CAN _USE_ THE MODEL (I.E. USE THE TRAINED MODEL TO DO PREDICTIONS WITHOUT MODIFYING THE MODEL)

If you came here directly without executing any of the lines above (except the imports), 
then you need to execute this:

This is the same as 2 cells above (but here in case you skipped it)

In [2]:
modelName = "reba3"
modelVersion = 2
modelName = "model-"+modelName+'-'+str(modelVersion)

In [3]:
# first load the model definition (and also secretly compile the model)
model = load_model(modelName+'-full.h5')


Now just to understand the data format, 
show the first row of the test dataset (that we use here to show how predictions are done)

In [10]:
X_test[0:2] # note: this is different from X_test[0] (the former is a matrix, the latter a vector)

array([[ -5.31099815e-01,   2.76901169e-01,  -2.52226559e-02,
         -7.26847457e-02,   1.01047846e+00,  -7.26118031e-01,
         -2.94650834e-01,  -1.08585917e+00,   9.35178197e-01,
         -5.88652304e-01,   6.65678763e-01,  -1.25610261e+00,
          2.13013092e+00,  -2.52756582e+00,   1.82643932e-01,
         -3.09919107e-01,   4.51483315e-01,   5.88298375e-01,
         -3.72508163e-01,   4.98656458e-02],
       [  5.39358420e-01,  -3.64856492e-01,   8.57235816e-02,
          4.93039267e-01,   1.56382459e+00,  -9.05761018e-02,
         -1.34786394e+00,  -1.06783212e+00,   1.40285548e+00,
          1.17479408e+00,   1.37042940e+00,   5.64028770e-01,
          1.12750297e+00,  -1.38838844e+00,   1.18513457e-03,
         -4.01957439e-01,   6.23303325e-02,   6.61052743e-01,
          4.15961789e-01,  -5.07119290e-01]])

this uses the model to predict the output for the first element in the test set

In [14]:
prediction = model.predict([X_test[0:1]])
print (np.around(prediction, 2))
print (" ")
print ("REBA:",int(np.argmax(prediction))+1)

[[ 0.          0.          0.          0.          0.          0.01        0.01
   0.25        0.73000002  0.        ]]
 
REBA: 9


...and this is the actual test set output (what the above output should be as close as possible to)

In [15]:
print (Y_test[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
